In [1]:
# Disable warnings in the notebook to maintain clean output cells
import warnings
warnings.filterwarnings('ignore')

# Import necessary libraries
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import yaml
from PIL import Image
from ultralytics import YOLO
from IPython.display import Video

In [2]:
best_model_path = 'D:/automatic_licence_recognition/YOLOv8_Traffic_Density_Estimation/models/best.pt'

# Load the best model weights into the YOLO model
best_model = YOLO(best_model_path)

In [3]:
dataset_video_path = 'D:/automatic_licence_recognition/YOLOv8_Traffic_Density_Estimation/data/test_videos/test2.mp4'
output_video = 'D:/automatic_licence_recognition/YOLOv8_Traffic_Density_Estimation/data/test_videos/test2_out2.avi'

In [ ]:
from utils.plots import Annotator
from pyimagesearch.centroidtracker import CentroidTracker
ct = CentroidTracker()

cap = cv2.VideoCapture(dataset_video_path)
ret, frame = cap.read()
print(frame.shape)

# Define the threshold for considering traffic as heavy
heavy_traffic_threshold = 10

# Define the vertices for the quadrilaterals
# vertices1 = np.array([(465, 350), (609, 350), (510, 630), (2, 630)], dtype=np.int32)

#video test2
vertices1 = np.array([(1387, 802), (425, 1865), (1735, 1852), (1675, 792)], dtype=np.int32)
vertices2 = np.array([(1862, 777), (2085, 760), (3790, 1785), (2657, 1870)], dtype=np.int32)

# video toulouse.MOV
# vertices1 = np.array([(1617, 1367), (1872, 1365), (1805, 1707), (1372, 1680)], dtype=np.int32)
# vertices2 = np.array([(1927, 1360), (2157, 1362), (2327, 1705), (1895, 1700)], dtype=np.int32)


# Define the vertical range for the slice and lane threshold
x1, x2 = min(vertices1[:,1]), max(vertices1[:,1])
lane_threshold = int((max(vertices1[:,0]) + min(vertices2[:,0]))/2)
print(x1, x2)
print('lane_threshold: ', lane_threshold)

# Define the positions for the text annotations on the image
text_position_left_lane = (10, 50)
intensity_position_left_lane = (10, 150)
frame_width = frame.shape[1]  # 获取图像宽度
right_text_x = frame_width - 1000

text_position_right_lane = (right_text_x, 50)
intensity_position_right_lane = (right_text_x, 150)


# Define font, scale, and colors for the annotations
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 2
font_color = (0, 0, 0)    # black color for text
# background_color = (0, 0, 255)  # Red background for text
background_color = (0, 255, 255) 
# Open the video


# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

# Read until video is completed

### wrong way detection parameters

FRAME_WIDTH = frame.shape[1]
FRAME_HEIGHT = frame.shape[0]
ROI_MIN = int(FRAME_HEIGHT*0.4)   #change the ROI dimension according to your camera angle
ROI_MAX = int(FRAME_HEIGHT*0.7)






previous_positions = {}
while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    rects = []
    if ret:

        cv2.line(frame, (50, ROI_MIN), (50, ROI_MAX), (255,0,255), 3)  # 左侧竖线左移
        cv2.line(frame, (FRAME_WIDTH - 50, ROI_MIN), (FRAME_WIDTH - 50, ROI_MAX), (255,0,255), 3)  # 右侧竖线保持不变
        cv2.line(frame, (50, ROI_MIN), (FRAME_WIDTH-50, ROI_MIN), (255,0,255), 3)  # 上边横线左移
        cv2.line(frame, (50, ROI_MAX), (FRAME_WIDTH-50, ROI_MAX), (255,0,255), 3)  # 下边横线左移



        # Create a copy of the original frame to modify
        detection_frame = frame.copy()
    
        # Black out the regions outside the specified vertical range
        detection_frame[:x1, :] = 0  # Black out from top to x1
        detection_frame[x2:, :] = 0  # Black out from x2 to the bottom of the frame
        
        # Perform inference on the modified frame
        results = best_model.predict(detection_frame, imgsz=640, conf=0.4)
        processed_frame = results[0].plot(line_width=5)
        
        # Restore the original top and bottom parts of the frame
        processed_frame[:x1, :] = frame[:x1, :].copy()
        processed_frame[x2:, :] = frame[x2:, :].copy()        
        
        # Draw the quadrilaterals on the processed frame
        cv2.polylines(processed_frame, [vertices1], isClosed=True, color=(0, 255, 0), thickness=5)
        cv2.polylines(processed_frame, [vertices2], isClosed=True, color=(255, 0, 0), thickness=5)
        
        # Retrieve the bounding boxes from the results
        bounding_boxes = results[0].boxes

        # Initialize counters for vehicles in each lane
        vehicles_in_left_lane = 0
        vehicles_in_right_lane = 0

        # Loop through each bounding box to count vehicles in each lane
        for box in bounding_boxes.xyxy:
            # Check if the vehicle is in the left lane based on the x-coordinate of the bounding box
            if box[0] < lane_threshold:
                vehicles_in_left_lane += 1
            else:
                vehicles_in_right_lane += 1
                
            x1_ww, y1_ww, x2_ww, y2_ww = map(int, box[:4])  # 获取 bounding box 的坐标
            # vehicle_id = f"{x1_ww}_{y1_ww}_{x2_ww}_{y2_ww}"  # 简单构造车辆 ID（可改为更稳定的跟踪方法）
            
            box_dimension = (x1_ww, y1_ww, x2_ww, y2_ww)
            box_center = (y1_ww+y2_ww)/2
            if (ROI_MIN <= box_center <= ROI_MAX):
                rects.append(box_dimension)
        objects, CY1, CY2 = ct.update(rects)  #send the box to tracker

        for (objectID, centroid) in objects.items():
            cy1=list(CY1.values())[objectID]
            cy2=list(CY2.values())[objectID]

            if (cy2<=cy1):                  #check whether the vehicle is incoming or outgoing
                text = "{}".format('right')
                #text = "ID: {}".format(objectID)
                cv2.putText(processed_frame, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255, 0), 4)
                cv2.circle(processed_frame, (centroid[0], centroid[1]), 3, (0, 0, 255), -1)
            else:
                text = "{}".format('wrong')
                cv2.putText(processed_frame, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0, 255), 4)
                cv2.circle(processed_frame, (centroid[0], centroid[1]), 3, (0, 0, 255), -1)





        # Determine the traffic intensity for the left lane
        traffic_intensity_left = "Heavy" if vehicles_in_left_lane > heavy_traffic_threshold else "Smooth"
        # Determine the traffic intensity for the right lane
        traffic_intensity_right = "Heavy" if vehicles_in_right_lane > heavy_traffic_threshold else "Smooth"


        # Add a background rectangle for the left lane vehicle count
        cv2.rectangle(processed_frame, (text_position_left_lane[0]-10, text_position_left_lane[1] - 50), 
                      (text_position_left_lane[0] + 800, text_position_left_lane[1] + 20), background_color, -1)

        # Add the vehicle count text on top of the rectangle for the left lane
        cv2.putText(processed_frame, f'Vehicles in Left Lane: {vehicles_in_left_lane}', text_position_left_lane, 
                    font, font_scale, font_color, 5, cv2.LINE_AA)

        # Add a background rectangle for the left lane traffic intensity
        cv2.rectangle(processed_frame, (intensity_position_left_lane[0]-10, intensity_position_left_lane[1] - 50), 
                      (intensity_position_left_lane[0] + 800, intensity_position_left_lane[1] + 20), background_color, -1)

        # Add the traffic intensity text on top of the rectangle for the left lane
        cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity_left}', intensity_position_left_lane, 
                    font, font_scale, font_color, 5, cv2.LINE_AA)

        # Add a background rectangle for the right lane vehicle count
        cv2.rectangle(processed_frame, (text_position_right_lane[0]-10, text_position_right_lane[1] - 50), 
                      (text_position_right_lane[0] + 800, text_position_right_lane[1] + 20), background_color, -1)

        # Add the vehicle count text on top of the rectangle for the right lane
        cv2.putText(processed_frame, f'Vehicles in Right Lane: {vehicles_in_right_lane}', text_position_right_lane, 
                    font, font_scale, font_color, 5, cv2.LINE_AA)

        # Add a background rectangle for the right lane traffic intensity
        cv2.rectangle(processed_frame, (intensity_position_right_lane[0]-10, intensity_position_right_lane[1] - 50), 
                      (intensity_position_right_lane[0] + 800, intensity_position_right_lane[1] + 20), background_color, -1)

        # Add the traffic intensity text on top of the rectangle for the right lane
        cv2.putText(processed_frame, f'Traffic Intensity: {traffic_intensity_right}', intensity_position_right_lane, 
                    font, font_scale, font_color, 5, cv2.LINE_AA)

        # Write the processed frame to the output video
        out.write(processed_frame)
        
        # Uncomment the following 3 lines if running this code on a local machine to view the real-time processing results
        # cv2.imshow('Real-time Analysis', processed_frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):  # Press Q on keyboard to exit the loop
        #     break
    else:
        break

# Release the video capture and video write objects
cap.release()
out.release()

(2160, 3840, 3)
792 1865
lane_threshold:  1798

0: 384x640 3 Vehicles, 79.0ms
Speed: 9.6ms preprocess, 79.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Vehicles, 86.0ms
Speed: 2.9ms preprocess, 86.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Vehicles, 64.3ms
Speed: 2.6ms preprocess, 64.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Vehicles, 53.7ms
Speed: 4.1ms preprocess, 53.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Vehicles, 78.4ms
Speed: 6.9ms preprocess, 78.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Vehicles, 52.4ms
Speed: 3.2ms preprocess, 52.4ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Vehicles, 80.6ms
Speed: 3.4ms preprocess, 80.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Vehicles, 52.8ms
Speed: 2.4ms preprocess, 5

In [1]:
import cv2
import numpy as np

# 读取视频的第一帧
cap = cv2.VideoCapture("D:/automatic_licence_recognition/YOLOv8_Traffic_Density_Estimation/data/test_videos/test2.mp4")
ret, frame = cap.read()
cap.release()

if not ret:
    print("无法读取视频帧")
    exit()

# 定义缩放比例（比如 50%）
scale = 0.4
resized_frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)

# 存储用户点击的顶点
vertices = []

def click_event(event, x, y, flags, param):
    """ 鼠标点击事件，存储顶点坐标 """
    if event == cv2.EVENT_LBUTTONDOWN:
        # 记录缩放后的坐标
        vertices.append((x, y))
        # 在缩小的图像上画点
        cv2.circle(resized_frame, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow("Select Points", resized_frame)

# 创建窗口并绑定鼠标点击事件
cv2.namedWindow("Select Points")
cv2.setMouseCallback("Select Points", click_event)

# 显示缩小的帧
cv2.imshow("Select Points", resized_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 还原回原始坐标
original_vertices = [(int(x / scale), int(y / scale)) for x, y in vertices]

# 输出最终的四边形顶点
print("原始视频的顶点坐标:", original_vertices)



原始视频的顶点坐标: [(1387, 802), (425, 1865), (1735, 1852), (1675, 792), (1862, 777), (2085, 760), (3790, 1785), (2657, 1870)]


In [1]:
import numpy as np 
vertices1 = np.array([(1387, 797), (1670, 802), (1695, 1297), (957, 1267)], dtype=np.int32)

In [19]:
vertices1 = np.array([(1387, 797), (1670, 802), (1695, 1297), (957, 1267)], dtype=np.int32)
vertices2 = np.array([(1882, 795), (2120, 772), (2752, 1150), (2197, 1235)], dtype=np.int32)
print(vertices1[:,0])
print(max(vertices1[:,0]))
print(min(vertices2[:,0]))

[1387 1670 1695  957]
1695
1882


In [20]:
int((max(vertices1[:,0]) + min(vertices2[:,0]))/2)

1788